In [1]:
import codecs

markov_sentences = codecs.open('2_markov_sentences.txt', 'r', encoding='utf-8').read().split('\n')

print 'len(markov_sentences)', len(markov_sentences)

len(markov_sentences) 10001


In [2]:
import spacy
nlp = spacy.load('en')

print spacy.__version__

1.9.0


### Transform the generated sentences  . . . 

. . . into lists of tokens, because I want to . . . 

In [5]:
sentences_semantic_data = []

for s in markov_sentences:
    
    semantic_data = []
    
    doc = nlp(unicode(s))
    
    for c in doc.noun_chunks:
        
        if len(c) == 1 and c[0].pos_ in ['PRON', 'PROPN']:
            pass
        else:
            
            all_pos = []
            for t in c:
                all_pos.append(t.pos_)
            all_pos = list(set(all_pos))
            
            if len(all_pos) == 1 and all_pos[0] == 'PROPN':
                pass
            else:
                semantic_data.append(c.text)
                
    for t in doc:
        if t.pos_ == 'VERB':
            semantic_data.append(t.lemma_)
                
    sentences_semantic_data.append(semantic_data) 
    
print 'len(sentences_semantic_data)', len(sentences_semantic_data)

 len(sentences_semantic_data) 10001


### . . . topic model the generated sentences

In [29]:
from gensim import corpora, models, similarities
from gensim.models.wrappers import LdaMallet

N_TOPICS = 100

dictionary = corpora.Dictionary(sentences_semantic_data)
corpus = [dictionary.doc2bow(text) for text in sentences_semantic_data]

path_to_mallet_binary = '/home/spenteco/0/mallet-2.0.7/bin/mallet'

model = LdaMallet(path_to_mallet_binary, 
                  corpus=corpus, 
                  num_topics=N_TOPICS, 
                  id2word=dictionary,
                  optimize_interval=10)

In [30]:
lda_corpus = model[corpus]

### Get sentences for each topic


In [53]:
import re

all_topic_sentences = []

for topic in range(0, N_TOPICS):
    
    topic_sentences = []
    
    for a in range(0, len(lda_corpus)):
    
        if lda_corpus[a][0][0] == topic and lda_corpus[a][0][1] > 0.40:
        #if lda_corpus[a][0][0] == topic:
            topic_sentences.append(markov_sentences[a])
            
    all_topic_sentences.append(topic_sentences)
    
total_words = 0
total_sentences = 0

for a in range(0, len(all_topic_sentences)):
    
    text = ' '.join(all_topic_sentences[a])
    words = []
    for t in re.split('[^A-Za-z]', text):
        if t > '':
            words.append(t)
            
    total_words += len(words)
    total_sentences += len(all_topic_sentences[a])
    
    print a, 'len(all_topic_sentences[a])', len(all_topic_sentences[a]), len(words)
    
print
print 'total_words', total_words
print 'total_sentences', total_sentences

0 len(all_topic_sentences[a]) 15 422
1 len(all_topic_sentences[a]) 13 314
2 len(all_topic_sentences[a]) 15 446
3 len(all_topic_sentences[a]) 31 799
4 len(all_topic_sentences[a]) 30 784
5 len(all_topic_sentences[a]) 30 477
6 len(all_topic_sentences[a]) 23 494
7 len(all_topic_sentences[a]) 16 332
8 len(all_topic_sentences[a]) 25 494
9 len(all_topic_sentences[a]) 21 468
10 len(all_topic_sentences[a]) 20 529
11 len(all_topic_sentences[a]) 18 344
12 len(all_topic_sentences[a]) 46 1157
13 len(all_topic_sentences[a]) 15 379
14 len(all_topic_sentences[a]) 24 678
15 len(all_topic_sentences[a]) 23 539
16 len(all_topic_sentences[a]) 13 244
17 len(all_topic_sentences[a]) 16 374
18 len(all_topic_sentences[a]) 29 553
19 len(all_topic_sentences[a]) 58 1717
20 len(all_topic_sentences[a]) 22 556
21 len(all_topic_sentences[a]) 34 599
22 len(all_topic_sentences[a]) 34 813
23 len(all_topic_sentences[a]) 17 512
24 len(all_topic_sentences[a]) 40 689
25 len(all_topic_sentences[a]) 37 753
26 len(all_topic_sen

### Just checking . . . do I actually have something?

In [55]:
import re
from collections import defaultdict, Counter
import nltk
from nltk.corpus import stopwords
 
sw = set(stopwords.words('english'))
    
for a in range(0, len(all_topic_sentences)):
    
    tokens = []
    for t in re.split('[^a-z]', ' '.join(all_topic_sentences[a]).lower()):
        if t > '' and t not in sw:
             tokens.append(t)
    
    token_count = defaultdict(int)
                
    for b in range(0, len(tokens) - 1):
        token_count[' '.join(tokens[b: b + 2])] += 1
                
    print
    print a
    print
    
    for w in Counter(token_count).most_common(5):
        print w[0], w[1]


0

west coast 6
dashes coast 3
lightning dashes 3
nasal drawl 2
year drives 2

1

rising degrees 5
vector rising 5
degrees vector 5
crawling things 3
things like 3

2

hurrah last 4
left alive 4
last man 4
man left 4
alive hurrah 4

3

wrist watch 9
black velvet 9
velvet space 4
interplanetary exchange 2
arrived bearing 2

4

hiding place 14
naked eye 6
hiding places 3
north america 2
cement walk 2

5

north pole 13
south pole 4
north america 3
reach north 3
attempts reach 3

6

awful lot 6
sir guy 2
leather socket 2
rouse pack 2
roused voice 2

7

south pole 5
gunpowder printing 3
printing press 3
press knows 2
cement roadway 2

8

early dawn 6
eventually advancing 3
tens thousands 3
san francisco 3
solar system 3

9

doctor corner 3
sort corn 3
corn doctor 3
land sunshine 3
spell binder 3

10

self sufficient 3
facility command 3
like arm 3
control self 3
self service 3

11

adult life 4
tabby adult 4
spoken every 2
successful communications 2
conveyed every 2

12

per cent 3
old ge

In [56]:
for a in range(0, len(all_topic_sentences)):
    
    f = codecs.open('raw_stories/' + str(a) + '.txt', 'w', encoding='utf-8')
    f.write('\n\n'.join(all_topic_sentences[a]))
    f.close()
    
print 'Done!'

Done!
